In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyccl as ccl
import scipy

In [ ]:
cosmo = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)

cosmo_no_lambda = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    w0 = 0., wa = 0,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)

cosmo_no_lambda_v2 = ccl.Cosmology(
    Omega_c=0.94, Omega_b=0.05,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)

cosmo_w0wa = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    w0=-0.8, wa=-1.0,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)


In [ ]:
n_obj = 1200
z_lens_mean = 0.74
z_lens_scale = 0.49
z_source_mean = 2.31
z_source_scale = 0.96

In [ ]:
np.log(z_lens_mean)

In [ ]:
z_lens_dist = scipy.stats.lognorm(scale=z_lens_mean, s=z_lens_scale)
z_source_dist = scipy.stats.lognorm(scale=z_source_mean, s=z_source_scale)

In [ ]:
z_lens_vals = z_lens_dist.rvs(n_obj)
z_source_vals = z_source_dist.rvs(n_obj)
mask = z_source_vals - z_lens_vals > 0.1
z_lens_vals = z_lens_vals[mask]
z_source_vals = z_source_vals[mask]

In [ ]:
_ = plt.hist(z_lens_vals, bins=np.linspace(0, 6, 121), alpha=0.6, label="lens")
_ = plt.hist(z_source_vals, bins=np.linspace(0, 6, 121), alpha=0.6, label="source")
_ = plt.xlabel('z')
_ = plt.ylabel("Objects / [0.05]")

In [ ]:
_ = plt.scatter(z_lens_vals, z_source_vals)
_ = plt.xlim(0, 4)
_ = plt.ylim(0, 4)
_ = plt.xlabel(r"$z_{d}$")
_ = plt.ylabel(r"$z_{s}$")

In [ ]:
def z_to_a(z: np.ndarray) -> np.ndarray:
    return 1/(1+z)

def a_to_z(a: np.ndarray) -> np.ndarray:
    return 1/(1-a)
    
def ratio(cosmo, z_d: np.ndarray, z_s: np.ndarray) -> np.ndarray:
    a_d = z_to_a(z_d)
    a_s = z_to_a(z_s)
    chi_d = cosmo.comoving_radial_distance(a_d.ravel()).reshape(a_d.shape)
    chi_s = cosmo.comoving_radial_distance(a_s.ravel()).reshape(a_s.shape)
    chi_l = chi_s - chi_d
    rat = (1 + (chi_d/chi_l))/a_s
    return np.where(z_s-z_d>0.2, rat, 0)

def chiRatio(cosmo, z_d: np.ndarray, z_s: np.ndarray) -> np.ndarray:
    a_d = z_to_a(z_d)
    a_s = z_to_a(z_s)
    chi_d = cosmo.comoving_radial_distance(a_d.ravel()).reshape(a_d.shape)
    chi_s = cosmo.comoving_radial_distance(a_s.ravel()).reshape(a_s.shape)
    rat = chi_d/chi_s
    return np.where(z_s-z_d>0.2, rat, 0)

In [ ]:
vals = chiRatio(cosmo, z_lens_vals, z_source_vals)
vals_no_lambda = chiRatio(cosmo_no_lambda, z_lens_vals, z_source_vals)
vals_wowa = chiRatio(cosmo_w0wa, z_lens_vals, z_source_vals)

scatter = np.random.normal(0, 0.05, size=len(z_source_vals))

In [ ]:
vals_meas = vals*(1+scatter)
vals_meas_wowa = vals_wowa*(1+scatter)

In [ ]:
_ = plt.scatter(z_lens_vals, z_source_vals, c=vals_meas, norm='log')
_ = plt.xlim(0, 4)
_ = plt.ylim(0, 4)
_ = plt.colorbar()

In [ ]:
max_abs = np.fabs(vals_meas-vals_no_lambda).max()
_ = plt.scatter(z_lens_vals, z_source_vals, c=vals_meas-vals, s=4, vmin=-max_abs, vmax=max_abs)
_ = plt.xlim(0, 4)
_ = plt.ylim(0, 4)
_ = plt.colorbar()

In [ ]:
max_abs = np.fabs(vals_meas-vals_no_lambda).max()
_ = plt.scatter(z_lens_vals, z_source_vals, c=vals_meas-vals_no_lambda, s=5, vmin=-max_abs, vmax=max_abs)
_ = plt.xlim(0, 4)
_ = plt.ylim(0, 4)
_ = plt.colorbar()

In [ ]:
max_abs = np.fabs(vals_meas_wowa-vals).max()
_ = plt.scatter(z_lens_vals, z_source_vals, c=vals_meas_wowa-vals, s=5) #, vmin=-max_abs, vmax=max_abs)
_ = plt.xlim(0, 4)
_ = plt.ylim(0, 4)
_ = plt.colorbar()

In [ ]:
z_grid = np.linspace(0, 4, 401)
mesh = np.meshgrid(z_grid, z_grid)
r = ratio(cosmo, mesh[0], mesh[1])

In [ ]:
_ = plt.imshow(r, origin='lower', norm='log', extent=(z_grid[0], z_grid[-1], z_grid[0], z_grid[-1]), vmax=250)
_ = plt.colorbar(label=r"$D_{\Delta T}/D_{D}$")
_ = plt.xlabel(r'$z_{\rm d}$')
_ = plt.ylabel(r'$z_{\rm s}$')
_ = plt.title(r'$\Lambda CDM$')